# TechBytes: Using Python with Teradata Vantage
## Part 2: Data Exploration and Transformations - Building an ADS

The contents of this file are Teradata Public Content and have been released to the Public Domain.
Please see _license.txt_ file in the package for more information.

Alexander Kolovos and Tim Miller - May 2021 - v.2.0 \
Copyright (c) 2021 by Teradata \
Licensed under BSD

This TechByte demonstrates how to
* explore your data in teradataml via functions for summary statistics, feature characteristics, data type reporting, and creating plots from table data.
* use options to display the actual SQL query submitted by teradataml to the Database.
* transform your data features by using teradataml DataFrame methods, Vantage Analytics Library (VAL) functions, and SQLAlchemy expressions.
* persist a teradataml DataFrame as a table in the Database.

Contributions by:
- Alexander Kolovos, Sr Staff Software Architect, Teradata Product Engineering / Vantage Cloud and Applications.
- Tim Miller, Principal Software Architect, Teradata Product Management / Advanced Analytics.

### Initial Steps: Load libraries and create a Vantage connection

In [ ]:
# Load teradataml and dependency packages.
#
import os
import getpass as gp
from teradataml import create_context, remove_context, get_context
from teradataml import DataFrame, copy_to_sql, in_schema
from teradataml.options.display import display

# Import "valib" object from teradataml to execute Vantage Analytic Library functions.
# Set 'configure.val_install_location' to the database name where Vantage analytic library functions are installed.
from teradataml.analytics.valib import *
from teradataml.options.configure import configure
from teradataml.dataframe.sql_functions import case
configure.val_install_location = "TRNG_XSP"

from sqlalchemy.sql.expression import select, or_, extract, text, join, case as case_when
from sqlalchemy import func

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Specify a Teradata Vantage server to connect to. In the following statement, 
# replace the following argument values with strings as follows:
# <HOST>   : Specify your target Vantage system hostname (or IP address).
# <UID>    : Specify your Database username.
# <PWD>    : Specify your password. You can also use encrypted passwords via
#            the Stored Password Protection feature.
#con = create_context(host = <HOST>, username = <UID>, password = <PWD>, 
#                     database = <DB_Name>, "temp_database_name" = <Temp_DB_Name>)
#
con = create_context(host = "<Host_Name>", username = "<Username>",
                            password = gp.getpass(prompt='Password:'), 
                            logmech = "LDAP", database = "TRNG_TECHBYTES",
                            temp_database_name = "<Database_Name>")

### 1. Database tables in teradataml

The following data have been used for the present TechBytes series on Analytics with R and Python. They reside in the TRNG_XSP database on the Transcend server. It is a simplistic fictitious dataset of banking Customers (10K-ish rows), Accounts (20K-ish rows) and Transactions (1M-ish rows). The 3 subsets are related to each other in the following ways:

![TWMDemoDataModel](Inputs/Plots/DemoData.png)

In [ ]:
# Create DataFrames for the Customer, Accounts and Transactions tables in the
# Vantage Advanced SQL Engine. Using to_pandas() for a cleaner display format.
# Note: to_pandas() converts a teradataml DataFrame to a pandas DataFrame. To
#       this, it transfers all data of a table to the client; use with caution.
# Note: in_schema() argument can read data from nondefault databases, as long
#       as you have read permissions for the specified database.
#
tdCustomer = DataFrame(in_schema("TRNG_XSP", "Customer")) 
tdCustomer.to_pandas().head(5)

In [ ]:
tdAccounts = DataFrame(in_schema("TRNG_XSP", "Accounts"))
tdAccounts.to_pandas().head(5)

In [ ]:
tdTransactions = DataFrame(in_schema("TRNG_XSP", "Transactions"))
tdTransactions.to_pandas().head(5)

### 2. Data Exploration

#### 2.1. Table info and summary statistics 

In [ ]:
# Inquire the data types of a table behind a teradataml DataFrame.
#
print(tdCustomer.tdtypes)

In [ ]:
# Use the Values function from VAL to inspect feature characteristics in the
# tdCustomer teradataml dataset.
#
tdCustomer_values = valib.Values(data = tdCustomer, columns=["all"])
tdCustomer_values.result.to_pandas()

In [ ]:
# Use the Statistics function from VAL to compute summary statistics about the
# tdCustomer teradataml dataset.
tdCustomer_stats = valib.Statistics(data=tdCustomer, columns="allnumericanddate")
tdCustomer_stats.result.to_pandas()

In [ ]:
# Use the teradataml option to print the SQL code of calls to Advanced SQL
# or ML Engines analytic functions.
#
display.print_sqlmr_query = True

# Use the show_query() method in analytic functions to display the SQL code
# that teradataml pushes to the Database for execution. For example:
#
valib.Values(data = tdCustomer, columns=["all"]).show_query()

#### 2.2. Plots from table data

In [ ]:
# Histogram of income
#
tdCustomer_hist_pd = tdCustomer[tdCustomer.income != None].to_pandas()
counts, bins = np.histogram(tdCustomer_hist_pd.income, bins = range(0, int(round(tdCustomer_hist_pd.income.max())), 10000))
bins = 0.5 * (bins[:-1] + bins[1:])
fig = px.bar(x = bins, y = counts)
fig.update_layout(height = 400, title = "Histogram of Income Distribution")
fig.update_xaxes(tickangle = 0, title = "Income")   
fig.update_yaxes(title = "Count")
fig.show()

In [ ]:
# Bar plot: Number of customers state-wise
#
tdCustomer_bar_pd = tdCustomer.groupby("state_code").count().select(['state_code','count_cust_id']).sort(['state_code']).to_pandas()
index = np.arange(tdCustomer_bar_pd.shape[0])
fig = px.bar(x = index, y = tdCustomer_bar_pd['count_cust_id'])
fig.update_layout(height = 400, title = "Bar plot of Customers per State")
fig.update_xaxes(tickangle = 0, title = "State ID")   
fig.update_yaxes(title = "Count")
fig.show()

#plt.bar(index, tdCustomer_bar_pd['count_cust_id'])
#plt.xlabel("State ID")
#plt.ylabel("No of customers")
#plt.xticks(index, tdCustomer_bar_pd['state_code'], rotation=30)
#plt.title("State-wise customers")
#plt.show()

### 2.3. Data Transformation

In [ ]:
# First, grab the customer demographic variables and create indicator variables
# for gender, marital_status and state_code (we consider both a classification
# into the top 6 states and jointly the rest as "OTHER", as well as creating
# indicator columns for each of the top 6 states).
# 
# For this task, we illustrate data manipulation with the use of SQLAlchemy.
# Initially we assemble the list of columns to be projected in SQL. We build it
# by using the SQLAlchemy function case_when() and objects 'Column' and 'Case'.
#
cust_select_query_column_projection = [
    tdCustomer.cust_id.expression,
    tdCustomer.income.expression,
    tdCustomer.age.expression,
    tdCustomer.years_with_bank.expression.label("tot_cust_years"),
    tdCustomer.nbr_children.expression.label("tot_children"),
    case_when([(tdCustomer.gender.expression == "M", 1)], else_=0).expression.label("male_ind"),
    case_when([(tdCustomer.gender.expression == "F", 1)], else_=0).expression.label("female_ind"),
    case_when([(tdCustomer.marital_status.expression == "1", 1)], else_=0).expression.label("single_ind"),
    case_when([(tdCustomer.marital_status.expression == "2", 1)], else_=0).expression.label("married_ind"),
    case_when([(tdCustomer.marital_status.expression == "3", 1)], else_=0).expression.label("separated_ind"),
    case_when([(tdCustomer.state_code.expression == "CA", "CA"),
               (tdCustomer.state_code.expression == "NY", "NY"),
               (tdCustomer.state_code.expression == "TX", "TX"),
               (tdCustomer.state_code.expression == "IL", "IL"),
               (tdCustomer.state_code.expression == "AZ", "AZ"),
               (tdCustomer.state_code.expression == "OH", "OH")
              ], else_="OTHER").expression.label("state_code"),
    case_when([(tdCustomer.state_code.expression == "CA", 1)], else_=0).expression.label("ca_resident_ind"),
    case_when([(tdCustomer.state_code.expression == "NY", 1)], else_=0).expression.label("ny_resident_ind"),
    case_when([(tdCustomer.state_code.expression == "TX", 1)], else_=0).expression.label("tx_resident_ind"),
    case_when([(tdCustomer.state_code.expression == "IL", 1)], else_=0).expression.label("il_resident_ind"),
    case_when([(tdCustomer.state_code.expression == "AZ", 1)], else_=0).expression.label("az_resident_ind"),
    case_when([(tdCustomer.state_code.expression == "OH", 1)], else_=0).expression.label("oh_resident_ind")
                                      ]
#
# Build and run the SQL from the list, then use it to create a DataFrame.
#
cust = DataFrame.from_query(str(select(cust_select_query_column_projection).compile(compile_kwargs={"literal_binds": True})))
cust.to_pandas().head(5)

In [ ]:
# Next, get the accounts information. Create account indicators and calculate
# account balances for each acct_type. Assume the median value for NaN balance
# entries.
#
# For this task, we illustrate data manipulation with the use of VAL functions.
#
fn = FillNa(style = "literal", value = 0)

account_type_t = OneHotEncoder(values = {"CC":"cc_acct_ind", "CK":"ck_acct_ind", "SV":"sv_acct_ind"}, 
                               columns = "acct_type", fillna = fn)
fillna_t = FillNa(style = "median", columns = ["cust_id", "starting_balance", "ending_balance"])

acctObj = valib.Transform(data = tdAccounts,
                          one_hot_encode = [account_type_t],
                          fillna = fillna_t,
                          key_columns = "cust_id")

acct_bal = acctObj.result.starting_balance + acctObj.result.ending_balance
#
# Build a DataFrame upon conclusion of the transformations.
#
acct = acctObj.result.assign(
           cc_bal = case_when( [(acctObj.result.cc_acct_ind.expression == 1, acct_bal.expression)
                               ], else_ = 0 )
                            ).assign(
           ck_bal = case_when( [(acctObj.result.ck_acct_ind.expression == 1, acct_bal.expression)
                               ], else_ = 0 )
                            ).assign(
           sv_bal = case_when( [(acctObj.result.sv_acct_ind.expression == 1, acct_bal.expression)
                               ], else_ = 0 )
                            )
acct.to_pandas().head(5)

In [ ]:
# Next, work on the transactions information. Add columns that indicate the 
# quarter a transaction was made.
#
acct_mon = extract('month', tdTransactions.tran_date.expression).expression

trans = tdTransactions.assign(
    q1_trans = case( [(acct_mon ==  "1", 1), (acct_mon ==  "2", 1), (acct_mon ==  "3", 1)], else_ = 0 ),
    q2_trans = case( [(acct_mon ==  "4", 1), (acct_mon ==  "5", 1), (acct_mon ==  "6", 1)], else_ = 0 ),
    q3_trans = case( [(acct_mon ==  "7", 1), (acct_mon ==  "8", 1), (acct_mon ==  "9", 1)], else_ = 0 ),
    q4_trans = case( [(acct_mon == "10", 1), (acct_mon == "11", 1), (acct_mon == "12", 1)], else_ = 0 ),
                             )
trans.to_pandas().head(5)

In [ ]:
# Finally, we want to bring together the 3 transformed tables.
#
# Left-join the transformed Customer table to the transformed Account table.
#
cust_acct = cust.join(other = acct, how = "left", on = ["cust_id"],
                             lsuffix = "cust", rsuffix = "acct")
cust_acct.to_pandas().head(5)

In [ ]:
# Then, left-join the previous result to the transformed Transaction table.
#
acct_tran_amt = trans.principal_amt + trans.interest_amt

cust_acct_tran = cust_acct.join(other = trans, how = "left", on = ["acct_nbr"], 
                                lsuffix = "cu_ac", rsuffix = "trans"
                               ).assign(
    cc_tran_amt = case_when( [(cust_acct.cc_acct_ind.expression == 1, acct_tran_amt.expression)
                             ], else_=0 )
                               ).assign(
    ck_tran_amt = case_when( [(cust_acct.ck_acct_ind.expression == 1, acct_tran_amt.expression)
                             ], else_=0 )
                               ).assign(
    sv_tran_amt = case_when( [(cust_acct.ck_acct_ind.expression == 1, acct_tran_amt.expression)
                             ], else_=0 )
                               )
cust_acct_tran.to_pandas().head(5)

### 2.4. Building an Analytic Data Set (ADS)

In [ ]:
# Finally, aggregate and roll up by 'cust_id' all variables to produce the
# Analytic Data Set (ADS) we will be using for our analyses.
#
ADS_Py = cust_acct_tran.groupby("cust_cust_id").agg(
    {
        "income"          : "max",
        "age"             : "max",
        "tot_cust_years"  : "max",
        "tot_children"    : "max",
        "female_ind"      : "max",
        "single_ind"      : "max",
        "married_ind"     : "max",
        "separated_ind"   : "max",
        "state_code"      : "max",
        "ca_resident_ind" : "max",
        "ny_resident_ind" : "max",
        "tx_resident_ind" : "max",
        "il_resident_ind" : "max",
        "az_resident_ind" : "max",
        "oh_resident_ind" : "max",
        "ck_acct_ind"     : "max",
        "sv_acct_ind"     : "max",
        "cc_acct_ind"     : "max",
        "ck_bal"          : "mean",
        "sv_bal"          : "mean",
        "cc_bal"          : "mean",
        "ck_tran_amt"     : "mean",
        "sv_tran_amt"     : "mean",
        "cc_tran_amt"     : "mean",
        "q1_trans"        : "sum",
        "q2_trans"        : "sum",
        "q3_trans"        : "sum",
        "q4_trans"        : "sum"
    }
)

# Rename columns

columns = ['cust_id','income','age','tot_cust_years','tot_children',
    'female_ind', 'single_ind', 'married_ind', 'separated_ind', 'state_code',
    'ca_resident_ind', 'ny_resident_ind', 'tx_resident_ind','il_resident_ind',
    'az_resident_ind', 'oh_resident_ind', 'ck_acct_ind', 'sv_acct_ind',
    'cc_acct_ind', 'ck_avg_bal', 'sv_avg_bal', 'cc_avg_bal', 'ck_avg_tran_amt',
    'sv_avg_tran_amt', 'cc_avg_tran_amt', 'q1_trans_cnt', 'q2_trans_cnt',
    'q3_trans_cnt','q4_trans_cnt']

ADS_Py = ADS_Py.assign(drop_columns = True,
                       cust_id         = ADS_Py.cust_cust_id,
                       income          = ADS_Py.max_income,
                       age             = ADS_Py.max_age,
                       tot_cust_years  = ADS_Py.max_tot_cust_years,
                       tot_children    = ADS_Py.max_tot_children,
                       female_ind      = ADS_Py.max_female_ind,
                       single_ind      = ADS_Py.max_single_ind,
                       married_ind     = ADS_Py.max_married_ind,
                       separated_ind   = ADS_Py.max_separated_ind,
                       state_code      = ADS_Py.max_state_code,
                       ca_resident_ind = ADS_Py.max_ca_resident_ind,
                       ny_resident_ind = ADS_Py.max_ny_resident_ind,
                       tx_resident_ind = ADS_Py.max_tx_resident_ind,
                       il_resident_ind = ADS_Py.max_il_resident_ind,
                       az_resident_ind = ADS_Py.max_az_resident_ind,
                       oh_resident_ind = ADS_Py.max_oh_resident_ind,
                       ck_acct_ind     = ADS_Py.max_ck_acct_ind,
                       sv_acct_ind     = ADS_Py.max_sv_acct_ind,
                       cc_acct_ind     = ADS_Py.max_cc_acct_ind,
                       ck_avg_bal      = ADS_Py.mean_ck_bal,
                       sv_avg_bal      = ADS_Py.mean_sv_bal,
                       cc_avg_bal      = ADS_Py.mean_cc_bal,
                       ck_avg_tran_amt = ADS_Py.mean_ck_tran_amt,
                       sv_avg_tran_amt = ADS_Py.mean_sv_tran_amt,
                       cc_avg_tran_amt = ADS_Py.mean_cc_tran_amt,
                       q1_trans_cnt    = ADS_Py.sum_q1_trans,
                       q2_trans_cnt    = ADS_Py.sum_q2_trans,
                       q3_trans_cnt    = ADS_Py.sum_q3_trans,
                       q4_trans_cnt    = ADS_Py.sum_q4_trans
                      ).select(columns)

ADS_Py = ADS_Py.dropna()

# teradataml DataFrame creates views at the backend which are temporary.
# At the end of the context removal, all temporary table/views perish.
# For this reason, persist the output of ADS_Py as a table in the SQLE.
copy_to_sql(ADS_Py, table_name="ak_TBv2_ADS_Py", if_exists="replace")

In [ ]:
# Create a teradataml DataFrame and take a glimpse at it.
#
td_ADS_Py = DataFrame("ak_TBv2_ADS_Py")
td_ADS_Py.to_pandas().head(5)

In [ ]:
# Split the ADS into 2 samples, each with 60% and 40% of total rows.
# Use the 60% sample to train, and the 40% sample to test/score.
# Persist the samples as tables in the Database, and create DataFrames.
#
td_Train_Test_ADS = td_ADS_Py.sample(frac = [0.6, 0.4])

Train_ADS = td_Train_Test_ADS[td_Train_Test_ADS.sampleid == "1"]
copy_to_sql(Train_ADS, table_name="ak_TBv2_Train_ADS_Py", if_exists="replace")
td_Train_ADS = DataFrame("ak_TBv2_Train_ADS_Py")

Test_ADS = td_Train_Test_ADS[td_Train_Test_ADS.sampleid == "2"]
copy_to_sql(Test_ADS, table_name="ak_TBv2_Test_ADS_Py", if_exists="replace")
td_Test_ADS = DataFrame("ak_TBv2_Test_ADS_Py")

### End of session

In [ ]:
# Remove the context of present teradataml session and terminate the Python
# session. It is recommended to call the remove_context() function for session
# cleanup. Temporary objects are removed at the end of the session.
#
remove_context()